### Download Grammar

In [125]:
from CodeCheckList import loader

"""define language"""
python_language = "python"

languages = [python_language]

loader.download_grammars(languages)

/home/svelascodimate/miniconda3/envs/code-check-list/lib/python3.10/site-packages/CodeCheckList/grammars


### Load Model

In [126]:

"""define the model checkpoint"""
checkpoint = "huggingface/CodeBERTa-small-v1"

### Create Modules

In [127]:
from CodeCheckList.tokenizer import CodeTokenizer
from CodeCheckList.masker import Masker

#create code tokenizer 
bert_tokenizer = CodeTokenizer.from_pretrained(checkpoint, python_language)

#create code masker
code_masker = Masker(bert_tokenizer)

### Node Types

In [128]:
print(bert_tokenizer.node_types)

['>=', 'unary_operator', 'chevron', 'return_statement', 'pair', 'break', 'lambda_parameters', 'exec', 'case_clause', 'false', 'list', 'global', 'format_expression', 'import', '<>', 'block', 'tuple', 'except', 'for', '[', 'pass', 'format_specifier', '^', 'for_statement', 'typed_default_parameter', 'with_statement', 'case', 'assignment', '<<=', 'assert', 'list_splat_pattern', 'and', '<<', 'if_clause', '//', '%=', '(', 'exec_statement', 'ellipsis', 'if_statement', 'continue_statement', 'keyword_separator', ':', 'parenthesized_list_splat', 'finally', ',', 'nonlocal', 'binary_operator', 'module', 'try_statement', 'conditional_expression', 'with_clause', 'as_pattern', 'import_prefix', '/=', 'else', '__future__', 'list_pattern', '@', 'await', 'import_from_statement', '&=', ';', 'expression_list', 'is', '}', 'not', '->', 'augmented_assignment', 'set', 'wildcard_import', '/', '>', 'async', 'if', '^=', '<', 'or', 'interpolation', 'generator_expression', 'decorated_definition', '**', '**=', 'from

### Encodings

In [129]:
"""example source code"""

code = "def multiply_numbers(a,b):\n    return a*b"
#code = "def hello_world(a,b):\n    print('hello world')"
#code = "def __ordered_values_by_indexes(self, data, inds): \"\"\" Return values (intensities) by indexes. Used for multiscale graph cut. data = [[0 1 1], [0 2 2], [0 2 2]] inds = [[0 1 2], [3 4 4], [5 4 4]] return: [0, 1, 1, 0, 2, 0] If the data are not consistent, it will take the maximal value \"\"\" # get unique labels and their first indexes # lab, linds = np.unique(inds, return_index=True) # compute values by indexes # values = data.reshape(-1)[linds] # alternative slow implementation # if there are different data on same index, it will take # maximal value # lab = np.unique(inds) # values = [0]*len(lab) # for label in lab: # values[label] = np.max(data[inds == label]) # # values = np.asarray(values) # yet another implementation values = [None] * (np.max(inds) + 1) linear_inds = inds.ravel() linear_data = data.ravel() for i in range(0, len(linear_inds)): # going over all data pixels if values[linear_inds[i]] is None: # this index is found for first values[linear_inds[i]] = linear_data[i] elif values[linear_inds[i]] < linear_data[i]: # here can be changed maximal or minimal value values[linear_inds[i]] = linear_data[i] values = np.asarray(values) return values"
#code = "def __ordered_values_by_indexes(self, data, inds):  # get unique labels and their first indexes # lab, linds = np.unique(inds, return_index=True) # compute values by indexes # values = data.reshape(-1)[linds] # alternative slow implementation # if there are different data on same index, it will take # maximal value # lab = np.unique(inds) # values = [0]*len(lab) # for label in lab: # values[label] = np.max(data[inds == label]) # # values = np.asarray(values) # yet another implementation values = [None] * (np.max(inds) + 1) linear_inds = inds.ravel() linear_data = data.ravel() for i in range(0, len(linear_inds)): # going over all data pixels if values[linear_inds[i]] is None: # this index is found for first values[linear_inds[i]] = linear_data[i] elif values[linear_inds[i]] < linear_data[i]: # here can be changed maximal or minimal value values[linear_inds[i]] = linear_data[i] values = np.asarray(values) return values"
target_node_type = "identifier"

#encoding 
source_code_encoding = bert_tokenizer(code)

#masking
masked_code_encoding = code_masker(code, bert_tokenizer(code), bert_tokenizer.node_types.index(target_node_type))

assert len(source_code_encoding['input_ids']) == len(masked_code_encoding['input_ids'])

#masked code
masked_code = bert_tokenizer.tokenizer.decode(list(filter(lambda token_id: False if token_id == bert_tokenizer.tokenizer.bos_token_id or 
            token_id == bert_tokenizer.tokenizer.eos_token_id else True, masked_code_encoding['input_ids'])))

print(masked_code)

def<mask><mask><mask>(<mask>,<mask>):
    return<mask>*<mask>


### Code Prediction

In [130]:
from CodeCheckList.predictor import Predictor

predictor = Predictor.from_pretrained(checkpoint, bert_tokenizer)
predictions = predictor(masked_code_encoding, bert_tokenizer.tokenizer(code, return_tensors='pt'), 5)

### Evaluation

In [131]:
from CodeCheckList.masker import find_nodes

prediction_number = 0
print('------------- CODE -------------')
print(code)
print('\n---------- MASKED -------------')
print(masked_code)
print('\n--------- PREDICTED -----------')
predicted_code = predictions[prediction_number]
print(predicted_code)
print('\n--------- AST COMPARE -----------')
filtered_nodes = []
filtered_nodes_predict = []
find_nodes(bert_tokenizer.parser.parse(bytes(code, "utf8")).root_node, bert_tokenizer.node_types[bert_tokenizer.node_types.index(target_node_type)], filtered_nodes)
find_nodes(bert_tokenizer.parser.parse(bytes(predicted_code, "utf8")).root_node, bert_tokenizer.node_types[bert_tokenizer.node_types.index(target_node_type)], filtered_nodes_predict)
print(filtered_nodes)
print(filtered_nodes_predict)
#base the evaluation on size comparison




------------- CODE -------------
def multiply_numbers(a,b):
    return a*b

---------- MASKED -------------
def<mask><mask><mask>(<mask>,<mask>):
    return<mask>*<mask>

--------- PREDICTED -----------
def __function(name, value):
    return f*args

--------- AST COMPARE -----------
[<Node type=identifier, start_point=(0, 4), end_point=(0, 20)>, <Node type=identifier, start_point=(0, 21), end_point=(0, 22)>, <Node type=identifier, start_point=(0, 23), end_point=(0, 24)>, <Node type=identifier, start_point=(1, 11), end_point=(1, 12)>, <Node type=identifier, start_point=(1, 13), end_point=(1, 14)>]
[<Node type=identifier, start_point=(0, 4), end_point=(0, 14)>, <Node type=identifier, start_point=(0, 15), end_point=(0, 19)>, <Node type=identifier, start_point=(0, 21), end_point=(0, 26)>, <Node type=identifier, start_point=(1, 11), end_point=(1, 12)>, <Node type=identifier, start_point=(1, 13), end_point=(1, 17)>]
